In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
max_feature=5000
maxlen=100
(x_train, y_train), (x_test,y_test)=imdb.load_data(num_words=max_feature)

In [ ]:
x_train=pad_sequences(x_train,maxlen=maxlen)
x_test=pad_sequences(x_test,maxlen=maxlen)

In [ ]:
#convert sequences back into text
reverse_word_index=dict((value,key) for (key,value) in imdb.get_word_index().items())
def decode_review(text):
    return ' '.join([reverse_word_index.get(i-3, '?') for i in text])

x_train_text = [decode_review(x) for x in x_train]
x_test_text = [decode_review(x) for x in x_test]

tokenizer = Tokenizer(num_words=max_feature)
tokenizer.fit_on_texts(x_train_text)
x_train=pad_sequences(tokenizer.texts_to_sequences(x_train_text), maxlen=maxlen)
x_test=pad_sequences(tokenizer.texts_to_sequences(x_test_text), maxlen=maxlen)

In [ ]:
model = Sequential()
model.add(Embedding(max_feature, 128, input_length=maxlen))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)

In [ ]:
score, acc=model.evaluate(x_test, y_test, batch_size=32)
print('Test accuracy: ', acc)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_input(user_input):
    sequences = tokenizer.texts_to_sequences(user_input)
    return pad_sequences(sequences, maxlen=maxlen)

In [ ]:
def predict_sentiment(review):
    review = preprocess_input([review])
    prediction = model.predict(review)[0][0]
    if prediction >= 0.5:
        return "Positive"
    else:
        return "Negative"

In [ ]:
while True:
    user_review = input("Enter your movie review (or type 'quit' to exit): ")
    if user_review.lower() == 'quit':
        print("Thank you for using our service!")
        break
    else:
        sentiment = predict_sentiment(user_review)
        print("Predicted sentiment:", sentiment)